In [1]:
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import random
import sys
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

from pprint import pprint

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
    PfeifferConfig
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader
import torch

from pdb import set_trace
import transformers.adapters.composition as ac

from transformers.adapters.heads import ClassificationHead
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets, ClassLabel, Value

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {'cola': {'load_adapter': 'lingaccept/cola@ukp', 'adapter_config': 'pfeiffer'},
                # 'mnli'
                'mrpc': {'load_adapter': 'sts/mrpc@ukp',        'adapter_config': 'pfeiffer'},
                'qnli': {'load_adapter': 'nli/qnli@ukp',        'adapter_config': 'pfeiffer'},
                'qqp' : {'load_adapter': 'sts/qqp@ukp',         'adapter_config': 'pfeiffer'},
                'rte' : {'load_adapter': 'nli/rte@ukp',         'adapter_config': 'pfeiffer'},
                'sst2': {'load_adapter': 'sentiment/sst-2@ukp', 'adapter_config': 'pfeiffer'},
                'stsb': {'load_adapter': 'sts/sts-b@ukp',       'adapter_config': 'pfeiffer'},
                
                'rotten_tomatoes': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-rotten_tomatoes', 'adapter_config': 'pfeiffer'},
                'imdb': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-imdb', 'adapter_config': 'pfeiffer'},
                'yelp_polarity': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-yelp_polarity', 'adapter_config': 'pfeiffer'},
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda 1


In [2]:
# if len(sys.argv) - 1 != 2:
#     print('Argument error')
#     exit(1)

# _, arg1, arg2 = sys.argv

# if arg1 not in adapter_info or arg2 not in adapter_info:
#     print(f'No adapter named {arg1} or {arg2}')
#     exit(1)

# task_name_1 = arg1
# task_name_2 = arg2
task_list = ['imdb', 'rotten_tomatoes', 'sst2', 'yelp_polarity']
adapter_list = [adapter_info[adapter]['load_adapter'] for adapter in task_list]
print(adapter_list)

['AdapterHub/bert-base-uncased-pf-imdb', 'AdapterHub/bert-base-uncased-pf-rotten_tomatoes', 'sentiment/sst-2@ukp', 'AdapterHub/bert-base-uncased-pf-yelp_polarity']


In [3]:
task_name_str = f'gating'
model_name_or_path = 'bert-base-uncased'
pad_to_max_length = True
max_seq_length = 128
output_dir = os.path.join(data_dir, f'case2_sts_moeBaseline_tmp/{task_name_str}_{current_time}')

adapter_config_default = 'pfeiffer'

adapter_k = 2
noisy_gating = True

num_labels = 2

random_seed = 0

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(output_dir)

/home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626


In [4]:
raw_datasets_list = []
for task_name in task_list:
    raw_datasets_list.append(load_dataset(task_name))

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]

        if sentence1_key == 'sentence':
            examples['text'] = examples['sentence']
            del examples['sentence']
        if 'idx' in examples:
            del examples['idx']
        
        return result
        
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [6]:
def align_dataset_labels(dataset, task_name):
    if task_name in ['sst2']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    elif task_name in ['yelp_polarity']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    return dataset
    
dataset_list = [get_data(task_name, raw_datasets) for task_name, raw_datasets in zip(task_list, raw_datasets_list)]   

_train_dataset_list = [dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=random_seed) for dataset in dataset_list]

train_dataset_list = [align_dataset_labels(d['train'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]
valid_dataset_list = [align_dataset_labels(d['test'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]

train_dataset = concatenate_datasets(train_dataset_list)
valid_dataset = concatenate_datasets(valid_dataset_list)

eval_dataset_list = [dataset['validation'] if task_name not in eval_data_dict else dataset[eval_data_dict[task_name]] for task_name, dataset in zip(task_list, dataset_list)] 

In [7]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 528703
})

In [8]:
valid_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 132176
})

In [9]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

model.freeze_model(True)

loaded_adapters = []
for adapter in adapter_list:
    loaded_adapter = model.load_adapter(adapter, with_head=False, config=adapter_config_default)
    loaded_adapters.append(loaded_adapter)

model.active_adapters = ac.Parallel(*loaded_adapters)

model.init_gating_network(adapter_k, noisy_gating)

model.add_classification_head(task_name_str)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
imdb                     bottleneck          894,528       0.816       1       1
rotten_tomatoes          bottleneck          894,528       0.816       1       1
sst-2                    bottleneck          894,528       0.816       1       1
yelp_polarity            bottleneck          894,528       0.816       1       1
--------------------------------------------------------------------------------
Full model                               109,629,696     100.000               1


In [11]:
model.active_head

'gating'

In [12]:
for k, v in model.named_parameters():
    if 'heads' in k or 'gating' in k:
            pass
    else:
        v.requires_grad = False

In [13]:
for k, v in model.named_parameters():
    if v.requires_grad:
        print(k)

bert.encoder.layer.0.attention.output.gating_networks.gating.w_noise
bert.encoder.layer.0.attention.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.0.output.gating_networks.gating.w_noise
bert.encoder.layer.0.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.1.attention.output.gating_networks.gating.w_noise
bert.encoder.layer.1.attention.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.1.output.gating_networks.gating.w_noise
bert.encoder.layer.1.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.2.attention.output.gating_networks.gating.w_noise
bert.encoder.layer.2.attention.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.2.output.gating_networks.gating.w_noise
bert.encoder.layer.2.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.3.attention.output.gating_networks.gating.w_noise
bert.encoder.layer.3.attention.output.gating_networks.gating.w_gate.weight
bert.encoder.layer.3.output.gating_networks.gating.w

In [14]:
per_device_train_batch_size = 64
per_device_eval_batch_size = 1024
weight_decay = 0.0
learning_rate = 5e-4
num_train_epochs = 20
lr_scheduler_type = 'cosine'
warmup_ratio = 0.1
patience = 5
alpha_info = 0.2

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

In [15]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [16]:
training_args = TrainingArguments(
    report_to='all',
    remove_unused_columns=False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir="./logs",
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='steps',
    logging_strategy='steps',
    save_strategy='steps',
    eval_steps=2000,
    logging_steps=2000,
    save_steps=2000,
    save_total_limit=1,
    load_best_model_at_end = True,
    metric_for_best_model = 'loss'
)

loss_fct = CrossEntropyLoss()

def get_gating_data(model):
    gate_scores = []
    gate_losses = []
    for bert_layer in model.bert.encoder.layer:
        gate_score = bert_layer.output.gating_data.pop('gate_score')
        gate_loss = bert_layer.output.gating_data.pop('gate_loss')
        gate_scores.append(gate_score)
        gate_losses.append(gate_loss)

    return gate_scores, torch.stack(gate_losses, 0).mean(0)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop('labels')

        # Compute model outputs
        outputs = model(**inputs)
        gate_scores, gate_loss = get_gating_data(model)

        logits = outputs[0].logits
        
        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1)) + alpha_info * gate_loss

        return loss
        
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0.0
        total_preds = []
        total_logits = []
        total_labels = []
        total_eval_metrics = {}

        adapter_freq = np.array([[0] * len(adapter_list)] * len(model.bert.encoder.layer))
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs = model(**inputs)
                gate_scores, gate_loss = get_gating_data(model)

                logits = outputs[0].logits

            loss = loss_fct(logits.view(-1, num_labels), labels.view(-1)) + alpha_info * gate_loss

            total_eval_loss += loss.item()

            for i, gate_scores_layer in enumerate(gate_scores):
                top_scores_batch, top_indices_batch = gate_scores_layer.topk(1, dim=1)
                for top_indices in top_indices_batch:
                    for top_index in top_indices:
                        adapter_freq[i][top_index] += 1

            total_logits.extend(logits.detach().cpu().numpy())
            total_preds.extend(logits.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())

        average_eval_loss = total_eval_loss / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        num_eval_samples = len(dataloader.dataset)

        all_adapter_freq = np.round(adapter_freq / num_eval_samples, decimals=4)
        avg_adapter_freq = np.around(np.mean(adapter_freq, axis=0) / num_eval_samples, decimals=4)
        # first_adapter_freq = adapter_freq[0] / num_eval_samples

        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss, 
                              f'{metric_key_prefix}_accuracy': metrics['accuracy'],
                              f'{metric_key_prefix}_freq_avg': list(avg_adapter_freq),
                              f'{metric_key_prefix}_freq_all': [list(o) for o in all_adapter_freq]
                             }

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=total_labels, 
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)


trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

In [17]:
os.makedirs(output_dir, exist_ok=True)
train_result = trainer.train()
metrics = train_result.metrics

loss_history = {'max_seq_length': max_seq_length,
                'random_seed': random_seed,
                'lr': learning_rate,
                'warmup_ratio': warmup_ratio,
                'early_stopping_patience': patience,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs,
                'task_list': task_list,
                'adapter_list': adapter_list,
                'adapter_k': adapter_k,
                'noisy_gating': noisy_gating,
                'alpha_info': alpha_info}


with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(loss_history, f)

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"trained_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"trained_head/{task_name_str}"), task_name_str)

/home/jaehan/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 528703
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 165220
  Number of trainable parameters = 739586


Step,Training Loss,Validation Loss,Accuracy,Freq Avg,Freq All
2000,0.188000,0.106786,0.963715,"[0.0075, 0.0317, 0.0762, 0.8846]","[[0.0, 0.0, 0.0004, 0.9996], [0.0, 0.0, 0.0012, 0.9988], [0.0, 0.0, 0.0065, 0.9935], [0.0, 0.0, 0.0357, 0.9643], [0.0, 0.0, 0.0515, 0.9485], [0.0, 0.0, 0.0713, 0.9287], [0.0, 0.0, 0.0596, 0.9404], [0.0, 0.0, 0.0362, 0.9638], [0.0, 0.0, 0.0307, 0.9693], [0.0, 0.0, 0.0152, 0.9848], [0.01, 0.0, 0.0682, 0.9217], [0.0804, 0.38, 0.5379, 0.0018]]"
4000,0.113700,0.096564,0.967521,"[0.0163, 0.0276, 0.1208, 0.8353]","[[0.0, 0.0, 0.0567, 0.9433], [0.0, 0.0, 0.0782, 0.9218], [0.0, 0.0, 0.0544, 0.9456], [0.0, 0.0, 0.086, 0.914], [0.0, 0.0, 0.1403, 0.8597], [0.0, 0.0, 0.1348, 0.8652], [0.0, 0.0, 0.1307, 0.8693], [0.0, 0.0, 0.1116, 0.8884], [0.0, 0.0, 0.1151, 0.8849], [0.0, 0.0053, 0.1446, 0.8501], [0.0012, 0.0012, 0.1067, 0.8908], [0.1947, 0.3244, 0.2902, 0.1907]]"
6000,0.106800,0.093887,0.969389,"[0.0211, 0.048, 0.1472, 0.7836]","[[0.0, 0.0, 0.0736, 0.9264], [0.0, 0.0, 0.1361, 0.8639], [0.0, 0.0, 0.1213, 0.8787], [0.0, 0.0, 0.1238, 0.8762], [0.0, 0.0, 0.1368, 0.8632], [0.0, 0.0, 0.1363, 0.8637], [0.0, 0.0, 0.1243, 0.8757], [0.0, 0.0, 0.1217, 0.8783], [0.0, 0.0, 0.1191, 0.8809], [0.0202, 0.1902, 0.1637, 0.6259], [0.0499, 0.0494, 0.1256, 0.7751], [0.1836, 0.3367, 0.3846, 0.0951]]"
8000,0.107200,0.094339,0.968860,"[0.0281, 0.0464, 0.1458, 0.7797]","[[0.0, 0.0, 0.0756, 0.9244], [0.0, 0.0, 0.1168, 0.8832], [0.0, 0.0, 0.1379, 0.8621], [0.0, 0.0, 0.1233, 0.8767], [0.0, 0.0, 0.1208, 0.8792], [0.0, 0.0, 0.1333, 0.8667], [0.0, 0.0, 0.118, 0.882], [0.0, 0.0, 0.1108, 0.8892], [0.0, 0.0, 0.1177, 0.8823], [0.0969, 0.1316, 0.1803, 0.5912], [0.0539, 0.0745, 0.1515, 0.7201], [0.1861, 0.3507, 0.3635, 0.0997]]"
10000,0.105900,0.091717,0.970501,"[0.0116, 0.0642, 0.1418, 0.7824]","[[0.0, 0.0, 0.0753, 0.9247], [0.0, 0.0, 0.1084, 0.8916], [0.0, 0.0, 0.1178, 0.8822], [0.0, 0.0, 0.1233, 0.8767], [0.0, 0.0, 0.1269, 0.8731], [0.0, 0.0, 0.1351, 0.8649], [0.0, 0.0, 0.1243, 0.8757], [0.0, 0.0, 0.1208, 0.8792], [0.0002, 0.0006, 0.1243, 0.8749], [0.0469, 0.0932, 0.1789, 0.681], [0.0332, 0.2227, 0.1079, 0.6362], [0.0593, 0.4535, 0.3584, 0.1287]]"
12000,0.105700,0.091727,0.969790,"[0.0292, 0.0591, 0.1296, 0.7822]","[[0.0, 0.0, 0.1046, 0.8954], [0.0, 0.0, 0.1194, 0.8806], [0.0, 0.0, 0.1005, 0.8995], [0.0, 0.0, 0.0941, 0.9059], [0.0, 0.0, 0.1076, 0.8924], [0.0, 0.0, 0.1333, 0.8667], [0.0, 0.0006, 0.1172, 0.8823], [0.0, 0.0, 0.0858, 0.9142], [0.0008, 0.0021, 0.1149, 0.8822], [0.1293, 0.2815, 0.1275, 0.4617], [0.0314, 0.0764, 0.1042, 0.788], [0.1885, 0.3482, 0.346, 0.1173]]"
14000,0.104700,0.092515,0.970426,"[0.0256, 0.0458, 0.1567, 0.7719]","[[0.0, 0.0, 0.2134, 0.7866], [0.0, 0.0, 0.1727, 0.8273], [0.0, 0.0, 0.121, 0.879], [0.0, 0.0, 0.1166, 0.8834], [0.0, 0.0, 0.1308, 0.8692], [0.0, 0.0, 0.1265, 0.8735], [0.0, 0.0152, 0.1067, 0.878], [0.0, 0.0002, 0.0816, 0.9182], [0.0, 0.0001, 0.1066, 0.8933], [0.0248, 0.1448, 0.1777, 0.6526], [0.0596, 0.1119, 0.253, 0.5755], [0.2227, 0.2779, 0.2732, 0.2261]]"
16000,0.103200,0.091392,0.969979,"[0.0353, 0.0503, 0.1327, 0.7818]","[[0.0, 0.0, 0.0872, 0.9128], [0.0, 0.0, 0.111, 0.889], [0.0, 0.0002, 0.0888, 0.911], [0.0261, 0.0, 0.1111, 0.8628], [0.0, 0.0, 0.1279, 0.8721], [0.0, 0.0, 0.1175, 0.8825], [0.0, 0.0001, 0.1123, 0.8876], [0.0116, 0.0002, 0.0941, 0.8942], [0.0002, 0.0124, 0.1169, 0.8705], [0.051, 0.076, 0.1899, 0.6831], [0.0983, 0.2469, 0.1049, 0.5499], [0.2364, 0.2677, 0.3305, 0.1654]]"
18000,0.105800,0.091468,0.970131,"[0.0283, 0.0586, 0.1633, 0.7498]","[[0.0, 0.0, 0.1043, 0.8957], [0.0, 0.0, 0.1461, 0.8539], [0.0, 0.0, 0.1482, 0.8518], [0.0, 0.0, 0.1452, 0.8548], [0.0, 0.0, 0.1591, 0.8409], [0.0, 0.0, 0.1575, 0.8425], [0.0, 0.0002, 0.1171, 0.8827], [0.0007, 0.0, 0.1161, 0.8832], [0.0005, 0.0835, 0.1418, 0.7742], [0.0182, 0.1334, 0.2386, 0.6097], [0.1121, 0.1961, 0.191, 0.5008], [0.2086, 0.2893, 0.2949, 0.2072]]"
20000,0.103000,0.095314,0.970123,"[0.0351, 0.0548, 0.1486, 0.7615]","[[0.0, 0.0, 0.0717, 0.928

Saving model checkpoint to /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000/config.json
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000/generation_config.json
Model weights saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000/tokenizer_config.json
Special tokens file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sts_moeBaseline_tmp/gating_20231123-104626/checkpoint-2000/special_tokens_map.json
Saving model 

***** train metrics *****
  epoch                    =        6.54
  total_flos               = 222384486GF
  train_loss               =      0.1071
  train_runtime            =  6:22:57.47
  train_samples_per_second =     460.192
  train_steps_per_second   =       7.191


In [18]:
metrics_dict = {}
for task_name, eval_dataset in zip(task_list, eval_dataset_list):
    metrics = trainer.evaluate(eval_dataset=eval_dataset)
    pprint(metrics)
    metrics_dict[task_name] = metrics
trainer.save_metrics("eval", metrics_dict)

{'epoch': 6.54,
 'eval_accuracy': 0.8777599930763245,
 'eval_freq_all': [[0.0, 0.0, 0.0012, 0.9988],
                   [0.1041, 0.0, 0.049, 0.8469],
                   [0.1537, 0.0035, 0.0002, 0.8426],
                   [0.0, 0.0, 0.1715, 0.8285],
                   [0.0, 0.0, 0.6102, 0.3898],
                   [0.0749, 0.0, 0.5222, 0.4029],
                   [0.7437, 0.0001, 0.0056, 0.2506],
                   [0.0, 0.0, 0.0001, 0.9999],
                   [0.4618, 0.0128, 0.3036, 0.2218],
                   [0.8533, 0.0395, 0.0603, 0.0469],
                   [0.9414, 0.0008, 0.0329, 0.025],
                   [0.58, 0.0107, 0.2303, 0.179]],
 'eval_freq_avg': [0.3261, 0.0056, 0.1656, 0.5027],
 'eval_loss': 0.2890433233976364,
 'eval_runtime': 55.7522,
 'eval_samples_per_second': 448.413,
 'eval_steps_per_second': 0.448}
{'epoch': 6.54,
 'eval_accuracy': 0.9118198752403259,
 'eval_freq_all': [[0.0, 0.0, 0.7205, 0.2795],
                   [0.0, 0.0, 0.8659, 0.1341],
              

In [19]:
# input('Remove files?\n')
# import shutil
# directory_path = output_dir
# shutil.rmtree(directory_path)

In [ ]:
import os
os._exit(00)